In [ ]:
import os
from pymilvus import MilvusClient  #Defingin a milvus client to create a local DB
import json
from sentence_transformers import SentenceTransformer

Define a object for Milvus Client; in our case it is RAGclient referrgin to the local db path

In [ ]:
RAGclient = MilvusClient("../DB/RAG.db")     # creating a local db file for milvus

Creating a collection in the Milvus DB object.


In [ ]:
RAGclient.create_collection(
    collection_name= "rag_collection",
    primary_field_name= "ID",
    vector_field_name= "ContentVector",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)

List the collections in the client object defined above; in this case it is by the name RAGclient

In [ ]:
#RAGclient.drop_collection(collection_name="rag_collection")
res = RAGclient.list_collections() 
print(res)


Get the count of existing rows from the db. (if any)

In [ ]:
rc = RAGclient.get_collection_stats(collection_name="rag_collection")
rows = rc['row_count']
rows

Uplaod the embeddgings in the local Milvus db

In [ ]:
def upload_embeddinf_in_db(embedding_file, collection_name):
      # Load data from JSON file
  with open(os.path.join(dir, embedding_file), "r") as f:
    data = json.load(f)

    insert_result = RAGclient.insert(collection_name=collection_name, data=data)
  
  return insert_result

#Usage 
dir = "../jsonfiles/" # Local path where the embeddign json files are saved
embedding_file = "data_with_embeddings.json"  # Replace with the embedding JSON file
collection_name = "rag_collection"

upload_embeddinf_in_db(embedding_file, collection_name)

Querrying the RAG_Collection DB

In [ ]:
model_name = "all-mpnet-base-v2"    # Using open source model from Hugging Face
modelPath = f"../model/{model_name}"      # Create a local path to store th model

model = SentenceTransformer(model_name)
model.save(modelPath)   # Saving the HF model in local path so that it can be used offline as well for generating embeddings

Loading the saved model from the local path


In [ ]:
# Load the SentenceTransformer model fromthe local path
model_name = "all-mpnet-base-v2"
modelPath = f"../model/{model_name}" 

model = SentenceTransformer(modelPath)
print(f"Model loaded - ",model)

Taking a question from the user with a input()

In [ ]:
# Gettign the query as an input from the user and create embedding of the query
q = input()
question = [q]
#question = ["What is the websote of thames water?"]
query_vector = model.encode(question).tolist()

search_result = RAGclient.search(
    collection_name="rag_collection",  # target collection
    data=query_vector,  # query vector from the user asked question
    limit=3,  # returning the top 3 entities
    output_fields=["Title", "Content"],  # specifies fields to be returned
)

search_result[0]

Displaying the citation for the article

In [ ]:
cit1 = search_result[0][0]["entity"]["Title"]
cit2 = search_result[0][1]["entity"]["Title"]

print(f"citation1: {cit1} \ncitation2: {cit2}")

Gathering the top 2 chunks returned by the index to create a single content to pass on to the LLM for RAG

In [ ]:
chunk1 = search_result[0][0]["entity"]["Content"]
chunk2 = search_result[0][1]["entity"]["Content"]
comprehensive_chunk = chunk1+chunk2
print(comprehensive_chunk)
#print(chunk1)


In [ ]:
%reset